In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
import re

In [27]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')


In [29]:
## 1. fuel_type null 값 채우기 (engine값으로)

def fill_fuel_type(df):
    # Hydrogen
    df.loc[df['engine'].str.contains('hydrogen', case=False, na=False), 'fuel_type'] = 'Hydrogen'
    
    # Plug-In Hybrid
    df.loc[df['engine'].str.contains('Plug-In', case=False) & df['fuel_type'].isna(), 'fuel_type'] = 'Plug-In Hybrid'
    
    # Hybrid
    df.loc[df['engine'].str.contains('Hybrid', case=False) & df['fuel_type'].isna(), 'fuel_type'] = 'Hybrid'
    df.loc[df['engine'].str.contains('Hybrid', case=False) & (df['fuel_type'] == '–'), 'fuel_type'] = 'Hybrid'
    df.loc[df['engine'].str.contains('Hybrid', case=False) & (df['fuel_type'] == 'not supported'), 'fuel_type'] = 'Hybrid'
    
    # Electric
    df.loc[df['engine'].str.contains('Electric', case=False) & df['fuel_type'].isna(), 'fuel_type'] = 'Electric'
    df.loc[df['engine'].str.contains('Standard Range Battery', case=False) & df['fuel_type'].isna(), 'fuel_type'] = 'Electric'
    df.loc[df['engine'].str.contains('Dual Motor - Standard', case=False) & df['fuel_type'].isna(), 'fuel_type'] = 'Electric'
    
    # Gasoline
    df.loc[df['engine'].str.contains('Gasoline', case=False) & df['fuel_type'].isna(), 'fuel_type'] = 'Gasoline'
    df.loc[df['engine'].str.contains('Gasoline', case=False) & (df['fuel_type'] == '–'), 'fuel_type'] = 'Gasoline'
    df.loc[df['engine'].str.contains('Gasoline', case=False) & (df['fuel_type'] == 'not supported'), 'fuel_type'] = 'Gasoline'
    df.loc[df['engine'].str.contains('GDI', case=False) & df['fuel_type'].isna(), 'fuel_type'] = 'Gasoline'
    
    return df

train = fill_fuel_type(train)
test = fill_fuel_type(test)

    # fuel_type null값 지우기
train = train[~(train['fuel_type'].isin(['–', 'not supported']) | train['fuel_type'].isnull())]

In [29]:
# 마력 (Horsepower):
# 마력은 차량의 성능을 나타내며, 일반적으로 마력이 높을수록 가격이 높아지는 경향이 있습니다.

# 배기량 (Displacement):
# 배기량은 엔진의 크기를 나타내며, 큰 배기량을 가진 차량은 일반적으로 성능이 좋고 가격이 더 높을 수 있습니다.

# 실린더 수 (Cylinder Count):
# 실린더 수는 엔진의 구성에 따라 성능과 연비에 영향을 미칠 수 있으며, 특정 브랜드나 모델에 따라 가격에 영향을 줄 수 있습니다.

# 연료 유형 (Fuel Type):
# 연료 유형(예: 가솔린, 디젤, 하이브리드 등)은 차량의 운영 비용과 성능에 영향을 미치며, 일부 연료 유형은 더 높은 가격을 받을 수 있습니다.

# 특수 기능 (Capability):
# 하이브리드 또는 터보차저와 같은 특수 기능은 성능을 개선할 수 있으며, 이러한 기능이 있는 차량은 일반적으로 더 높은 가격을 가질 수 있습니다.

In [ ]:
# 2. engine 열에 있는 정보 나눠서 쓰기


def parse_engine(engine_str):
    # 정규 표현식을 사용하여 마력, 배기량, 실린더 수를 추출 (연료 유형은 기존 데이터 사용)
    hp_match = re.search(r'(\d+\.?\d*)HP', engine_str)
    displacement_match = re.search(r'(\d+\.?\d*)L', engine_str)
    cylinder_match = re.search(r'(\d+)\s+Cylinder', engine_str)
    # fuel_match = re.search(r'(\w+ Fuel|Electric|Hybrid)', engine_str)

    # 마력, 배기량, 실린더 수 기본값 설정 (연료 유형은 기존 데이터 사용)
    horsepower = float(hp_match.group(1)) if hp_match else None
    displacement = float(displacement_match.group(1)) if displacement_match else None
    cylinder_count = int(cylinder_match.group(1)) if cylinder_match else None
    # fuel_type = fuel_match.group(1) if fuel_match else None

    return pd.Series([horsepower, displacement, cylinder_count])

# 엔진 특성 파싱 및 새로운 열 생성
train[['horsepower', 'displacement', 'cylinder_count']] = train['engine'].apply(parse_engine)
test[['horsepower', 'displacement', 'cylinder_count']] = test['engine'].apply(parse_engine)

# 엔진 열 삭제
train = train.drop(columns=['engine'])
test = test.drop(columns=['engine'])

In [34]:
## 3.치우친 값을 확인하여, 로그 변환 진행
train['log_price'] = np.log1p(train['price'])  # log1p는 log(1 + x)를 적용하여 0인 값을 처리

# # 로그 변환한 타겟으로 예측한 값이 log_predictions 라면
# original_predictions = np.expm1(log_predictions)
# 해서 역변환


## 4. clean_title 변수에서 null 값에 대해 "No" 로 수정하기

train['clean_title'] = train['clean_title'].fillna("No")
test['clean_title'] = test['clean_title'].fillna("No")

In [35]:
## 5. transmission 열에 있는 정보 나눠서 쓰기

def parse_transmission(transmission_str):
    # 변속기 유형 및 속도 추출
    if 'A/T' in transmission_str or 'Automatic' in transmission_str or 'CVT' in transmission_str:
        transmission_type = 'Automatic'
    elif 'M/T' in transmission_str or 'Manual' in transmission_str:
        transmission_type = 'Manual'
    else:
        transmission_type = None

    # 변속기 속도 추출
    speed_match = re.search(r'(\d+)-Speed', transmission_str)
    transmission_speed = speed_match.group(1) if speed_match else None

    # 특별 기능 추출
    special_feature_match = re.search(r'Dual Shift|Overdrive|Auto-Shift|DCT', transmission_str)
    special_feature = special_feature_match.group(0) if special_feature_match else None

    return pd.Series([transmission_type, transmission_speed, special_feature])

# transmission 열 파싱 및 새로운 열 생성
train[['transmission_type', 'transmission_speed', 'special_feature']] = train['transmission'].apply(parse_transmission)
test[['transmission_type', 'transmission_speed', 'special_feature']] = test['transmission'].apply(parse_transmission)

# transmission 열 삭제
train = train.drop(columns=['transmission'])
test = test.drop(columns=['transmission'])

In [37]:
# 6. transmission_type / transmission_speed null값 -> 같은 모델과 모델 연식으로 채우기

# train과 test를 결합
train['is_train'] = 1
test['is_train'] = 0
combined = pd.concat([train, test])

# transmission_type과 transmission_speed의 결측치 채우기
combined['transmission_type'] = combined.groupby(['model', 'model_year'])['transmission_type'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Unknown'))
combined['transmission_speed'] = combined.groupby(['model', 'model_year'])['transmission_speed'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 0))


# 7. horsepower, displacement, cylinder_count null값 -> 모델별 최빈값으로 채우기

# 모델별로 최빈값 계산
mode_values = combined.groupby('model')[['horsepower', 'displacement', 'cylinder_count']].agg(lambda x: x.mode()[0] if not x.mode().empty else None)

# 모델별 최빈값으로 결측치 채우기
for column in ['horsepower', 'displacement', 'cylinder_count']:
    combined[column] = combined[column].fillna(combined['model'].map(mode_values[column]))

# transmission_speed도 다시 int로 변환
combined['transmission_speed'] = combined['transmission_speed'].astype(int)



# train과 test로 다시 분리
train = combined[combined['is_train'] == 1].drop(columns=['is_train'])
test = combined[combined['is_train'] == 0].drop(columns=['is_train'])

In [42]:
# null 값 제거
train = train[train['accident'].notnull()]
train = train[train['displacement'].notnull()]
train = train[train['cylinder_count'].notnull()]
# 187716 개 -> 182058 개

# 0으로 null값 대체
train['special_feature'].fillna(0, inplace=True)
test['special_feature'].fillna(0, inplace=True)

# 마지막 null 값에 대해 모델명으로 검색해서 찾은 값 임의로 넣기
train.loc[train['model'] == 'Wagoneer Series II 4x4', 'horsepower'] = 392


test = test.drop(columns=['price'])
test = test.drop(columns=['log_price'])


In [62]:
train.to_csv('train22.csv', index=False)
test.to_csv('test22.csv', index=False)